In [3]:
#觀察範例，在房價預測中調整標籤編碼(Label Encoder) / 獨熱編碼 (One Hot Encoder) 方式，
#對於線性迴歸以及梯度提升樹兩種模型，何者影響比較大?
library(magrittr)
library(caret)
library(cattonum)

train=read.csv('C:\\Users\\coco40725\\Documents\\GitHub\\2nd-ML100Days\\data\\house.train.csv',sep=',',header=T)
test=read.csv('C:\\Users\\coco40725\\Documents\\GitHub\\2nd-ML100Days\\data\\house.test.csv',sep=',',header=T)

##extract categorical variable
col.type=rep(0,81)
for (i in 1:81) {
  col.type[i]=class(train[,i])
}
col.factor=which(col.type=='factor')


##only keep categorical column
train.cate=train[,c(col.factor,81)]
test.cate=test[,col.factor]

## covert NA into a level
for (j in 1:43) {
  train.cate[,j] %<>% addNA(.)
  
}


## linear regression+ lable coding
train.cate.label=train.cate
for (k in 1:43) {
  train.cate.label[,k] %<>% as.numeric(.)
}
t11=Sys.time()
train_control <- trainControl(method="cv", number=5)
cv1= train(SalePrice~., data=train.cate.label, trControl=train_control, method="lm")
t12=Sys.time()
time1=difftime(t11,t12,units='secs')
rmse1=cv1$results[2]
rsq1=cv1$results[3]
fit1=lm(SalePrice~.,data=train.cate.label)
summary(fit1)



## linear regression + one hot coding

train_control <- trainControl(method="cv", number=5)
t11=Sys.time()
cv2= train(SalePrice~., data=train.cate, trControl=train_control, method="lm")
t12=Sys.time()

time2=difftime(t11,t12,units='secs')
rmse2=cv2$results[2]
rsq2=cv2$results[3]
fit2=lm(SalePrice~.,data=train.cate)
summary(fit2)



library(dummies)

## gboost + lable coding
t11=Sys.time()
xgb.fit1= train(SalePrice~., data=train.cate.label, trControl=train_control, method="xgbTree")
t12=Sys.time()
gb.req1=mean(xgb.fit1$results$Rsquared)
gb.rmse1=mean(xgb.fit1$results$RMSE)
time.gb1=difftime(t11,t12,units='secs')

## gboost + one hot coding
train.dummy=catto_dummy(train.cate,colnames(train.cate)[-44])
train.dummy=cbind(train.dummy,SalePrice=train.cate[,44])
t11=Sys.time()
xgb.fit2= train(SalePrice~., data=train.dummy, trControl=train_control, method="xgbTree")
t12=Sys.time()
gb.req2=mean(xgb.fit2$results$Rsquared)
gb.rmse2=mean(xgb.fit2$results$RMSE)
time.gb2=difftime(t11,t12,units='secs')

######## 解析


result.fin=data.frame(type=c('reg+label','reg+one-hot','xgb+label','xgb+one-hot'),
                      rsq=as.numeric(c(rsq1,rsq2,gb.req1,gb.req2)),
                      rmse=as.numeric(c(rmse1,rmse2,gb.rmse1,gb.rmse2)),
                      time=-c(time1,time2,time.gb1,time.gb2))
result.fin

ERROR: Error in library(cattonum): there is no package called 'cattonum'


### linear regression and xgboost has better performance in one hot coding. However, it is obvious that we need to spend much time calculating cv results, since one hot coding increase a lot of dummy variables.

### since our data is nominal variable, it is inappropriate to use label coding in this data.

In [2]:
#鐵達尼號例題中，標籤編碼 / 獨熱編碼又分別對預測結果有何影響? (Hint : 參考今日範例)
#鐵達尼號例題中，標籤編碼 / 獨熱編碼又分別對預測結果有何影響? (Hint : 參考今日範例)
library(magrittr)
library(caret)
library(cattonum)
train=read.csv('C:\\Users\\coco40725\\Documents\\GitHub\\2nd-ML100Days\\data\\titanic_train.csv')
train=train[,-c(1,4,9)]

##extract categorical variable
col.type=rep(0,9)
for (i in 1:9) {
  col.type[i]=class(train[,i])
}
col.factor=which(col.type=='factor')



##only keep categorical column
train.cate=train[,c(col.factor,1)]

## covert NA into a level
for (j in 1:3) {
  train.cate[,j] %<>% addNA(.)
  
}

## linear regression+ lable coding
train.cate.label=train.cate
for (k in 1:3) {
  train.cate.label[,k] %<>% as.numeric(.)
}


t11=Sys.time()
train_control <- trainControl(method="cv", number=5)
cv1= train(as.factor(Survived)~., data=train.cate.label, family=binomial,trControl=train_control, method="glm")
t12=Sys.time()
time1=difftime(t11,t12,units='secs')
racc1=cv1$results[2]
rkappa1=cv1$results[3]

## linear regression + one hot coding
train_control <- trainControl(method="cv", number=5)
t11=Sys.time()
cv2= train(as.factor(Survived)~., data=train.cate, family=binomial,trControl=train_control, method="glm")
t12=Sys.time()

time2=difftime(t11,t12,units='secs')
racc2=cv2$results[2]
rkappa2=cv2$results[3]



## gboost + lable coding
t11=Sys.time()
xgb.fit1 =train(as.factor(Survived)~., data=train.cate.label, trControl=train_control, method="xgbTree")
t12=Sys.time()
gb.acc1=mean(xgb.fit1$results$Accuracy)
gb.kappa1=mean(xgb.fit1$results$Kappa)
time.gb1=difftime(t11,t12,units='secs')

## gboost + one hot coding
train.dummy=catto_dummy(train.cate,colnames(train.cate)[-4])
train.dummy=cbind(train.dummy,Survived=train.cate[,4])
t11=Sys.time()
xgb.fit2= train(as.factor(Survived)~., data=train.dummy, trControl=train_control, method="xgbTree")
t12=Sys.time()
gb.acc2=mean(xgb.fit2$results$Accuracy)
gb.kappa2=mean(xgb.fit2$results$Kappa)
time.gb2=difftime(t11,t12,units='secs')


result.fin=data.frame(type=c('reg+label','reg+one-hot','xgb+label','xgb+one-hot'),
                      acc=as.numeric(c(racc1,racc2,gb.acc1,gb.acc2)),
                      kappa=as.numeric(c(rkappa1,rkappa2,gb.kappa1,gb.kappa2)),
                      time=-c(time1,time2,time.gb1,time.gb2))
result.fin


ERROR: Error in library(cattonum): there is no package called 'cattonum'


### In titanic data which is a calssification problem,  logistic and xgboost still have slightly better performance in one hot coding, and still, one hot coding is more time consumming than label coding.